In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import Libraries

# DS 
import os
from itertools import islice
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
import seaborn as sns

# PLOTLY
import plotly.offline as po
import plotly.graph_objs as go

# SKLEARN
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import (GradientBoostingRegressor, 
                              GradientBoostingClassifier, 
                              AdaBoostClassifier,
                              AdaBoostRegressor,
                              RandomForestRegressor,
                              RandomForestClassifier)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, make_scorer, confusion_matrix, accuracy_score, plot_roc_curve
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence

import warnings
warnings.filterwarnings('ignore')

# Aesthetic Plots
import mplcyberpunk
plt.style.use('cyberpunk')
%matplotlib inline

In [3]:
from src.gradient_clean_data import *

In [4]:
# GLOBAL VARS
FSIZE = (12, 8)

In [5]:
# Load CSV
df = pd.read_csv('data/Train.csv')

In [6]:
# CSV Head
# df.head()

In [7]:
X, y = gradient_clean_Xy(df)

KeyError: 'Power Rating'

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
# Train Test Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)    

In [ ]:
# Create GradientBoostingRegressor Model
gb_model = GradientBoostingRegressor()

In [ ]:
# # Fit Model
# gb_model.fit(X_train, y_train)

In [ ]:
# # Find Feature Importances via GB Model

# # See and plot feature importances 
# feat_scores = pd.DataFrame({'Heavy Machinery Feature Importances' : gbmodel_grid_cv.feature_importances_},
#                            index=X.columns)
# feat_scores = feat_scores.sort_values(by='Heavy Machinery Feature Importances')
# feat_scores.plot(kind='barh', figsize=(10, 6))

In [ ]:
X["MachineHoursCurrentMeter"].value_counts()

In [ ]:
X['fiBaseModel'].value_counts()

In [ ]:
X['UsageBand'].unique()

In [ ]:
# Create Label Encoding Object
label_encode = LabelEncoder()

In [ ]:
label_encode.fit(X["UsageBand"])

In [ ]:
encoded = label_encode.transform(X["UsageBand"])

In [ ]:
label_encode.inverse_transform(encoded)

In [ ]:
X["fiModelDescriptor"].value_counts()

In [23]:
X["Ride_Control"].value_counts()

No                     77685
None or Unspecified    63116
Yes                     7805
Name: Ride_Control, dtype: int64